<a href="https://colab.research.google.com/github/rehabreda/NLP/blob/master/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving spa.txt to spa.txt
User uploaded file "spa.txt" with length 8152302 bytes


In [3]:
###import libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense ,LSTM ,Input ,Embedding 
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
import keras.backend as k
if len(k.tensorflow_backend._get_available_gpus())>0:
  from keras.layers import CuDNNLSTM as LSTM

In [0]:
## congiguration
batch_size=64
epochs=100
latent_dim=265
max_samples=10000
max_sequence_length=100
max_num_words=20000
embedding_dim=100

In [0]:
input_texts=[]
target_texts=[]
target_texts_input=[]

In [0]:
t=0
for line in open('spa.txt'):
  t+=1
  if t> max_samples:
    break
  if '\t' not in line:
    continue
  input_text,translation=line.rstrip().split('\t')
  target_text=translation +' <eos>'
  target_text_input='<sos> ' +translation
  
  input_texts.append(input_text)
  target_texts.append(target_text)
  target_texts_input.append(target_text_input)
  
  

In [0]:
###tokinizer input
tokenizer_inputs=Tokenizer(num_words=max_num_words)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences=tokenizer_inputs.texts_to_sequences(input_texts)

In [0]:
word2idx_inputs=tokenizer_inputs.word_index

In [0]:
max_len_input=max(len(s) for s in input_sequences)

In [0]:
##tokenizer targers
tokenizer_targets=Tokenizer(num_words=max_num_words,filters='')
tokenizer_targets.fit_on_texts(target_texts+target_texts_input)
target_sequences=tokenizer_targets.texts_to_sequences(target_texts)
target_sequences_input=tokenizer_targets.texts_to_sequences(target_texts_input)



In [0]:
word2idx_targets=tokenizer_targets.word_index

In [0]:
max_len_target=max(len(s) for s in target_sequences )

In [0]:
num_words_target=len(word2idx_targets)+1

In [0]:
###pad sequences
encoder_inputs=pad_sequences(input_sequences,maxlen=max_len_input)

decoder_inputs=pad_sequences(target_sequences_input,maxlen=max_len_target,padding='post')

decoder_outputs=pad_sequences(target_sequences,maxlen=max_len_target,padding='post')


In [16]:
## load pre-trained nodel
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [2]:
ls

glove.6B.100d.txt  sample_data/  spa.txt


In [17]:
word2vec={}

for line in open('glove.6B.100d.txt'):
  values=line.split()
  word=values[0]
  vec=np.asarray(values[1:],dtype='float32')
  word2vec[word]=vec
  
print('Found %s word vectors.' % len(word2vec))
  
  

Found 340790 word vectors.


In [0]:
## embedding matrix
num_words=min(max_num_words,len(word2idx_inputs)+1)
embedding_matrix=np.zeros((num_words,embedding_dim))

for word,idx in word2idx_inputs.items():
  if idx< max_num_words:
    embedding_vector=word2vec.get(word)
    if embedding_vector is not None:
      embedding_matrix[idx]=embedding_vector


In [0]:
## create embedding layer
embedding_layer=Embedding(num_words,embedding_dim,weights=[embedding_matrix],input_length=max_len_input)

In [0]:
decoder_targets_one_hot=np.zeros((len(input_texts),max_len_target,num_words_target))

In [0]:
for i ,s in enumerate(decoder_outputs):
  for t,word in enumerate(s):
    decoder_targets_one_hot[i,t,word]=1
    

In [22]:
## build model 
encoder_input_placeholder=Input(shape=(max_len_input,))
x=embedding_layer(encoder_input_placeholder)
encoder=LSTM(latent_dim,return_state=True)
encoder_output,h,c=encoder(x)
encoder_states=[h,c]

##decoder
decoder_input_placeholder=Input(shape=(max_len_target,))
decoder_embedding=Embedding(num_words_target,latent_dim)
decoder_output_x=decoder_embedding(decoder_input_placeholder)
decoder=LSTM(latent_dim,return_state=True,return_sequences=True)
decoder_output,_,_=decoder(decoder_output_x,initial_state=encoder_states)

decoder_dense=Dense(num_words_target,activation='softmax')
decoder_output=decoder_dense(decoder_output)
model=Model([encoder_input_placeholder,decoder_input_placeholder],decoder_output)                    

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.compile(
loss='categorical_crossentropy',
optimizer='rmsprop',
metrics=['accuracy']       
)

In [24]:
model.fit([encoder_inputs,decoder_inputs],decoder_targets_one_hot,batch_size=batch_size,epochs=epochs,validation_split=.2)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 11s 1ms/step - loss: 2.6202 - acc: 0.6605 - val_loss: 2.5767 - val_acc: 0.6606
Epoch 2/100
8000/8000 [==============================] - 8s 1ms/step - loss: 2.0101 - acc: 0.7152 - val_loss: 2.4134 - val_acc: 0.6746
Epoch 3/100
8000/8000 [==============================] - 8s 1ms/step - loss: 1.8047 - acc: 0.7364 - val_loss: 2.2502 - val_acc: 0.6893
Epoch 4/100
8000/8000 [==============================] - 8s 1ms/step - loss: 1.6440 - acc: 0.7549 - val_loss: 2.1652 - val_acc: 0.7068
Epoch 5/100
8000/8000 [==============================] - 8s 1ms/step - loss: 1.5151 - acc: 0.7711 - val_loss: 2.0454 - val_acc: 0.7215
Epoch 6/100
8000/8000 [==============================] - 8s 1ms/step - loss: 1.4038 - acc: 0.7832 - val_loss: 2.0169 - val_acc: 0.7273
Epoch 7/100
800

In [0]:
## make prediction  
## build sampling model 

## encoder
encoder_model = Model(encoder_input_placeholder, encoder_states)

decoder_input_h=Input(shape=(latent_dim,))
decoder_input_c=Input(shape=(latent_dim,))
decoder_states_input=[decoder_input_h,decoder_input_c]

decoder_input_signle=Input(shape=(1,))
decoder_input_signle_x=decoder_embedding(decoder_input_signle)
decoder_output,h,c=decoder(decoder_input_signle_x,initial_state=decoder_states_input)
decoder_states=[h,c]

decoder_output=decoder_dense(decoder_output)

decoder_model=Model([decoder_input_signle]+decoder_states_input,[decoder_output]+ decoder_states)




In [0]:
idx2word_inputs={v:k for k,v in word2idx_inputs.items()}
idx2word_targets={v:k for k,v in word2idx_targets.items()}

In [0]:
def decode_sequence(input_seq):
  states=encoder_model.predict(input_seq)
  target_seq=np.zeros((1,1))
  
  target_seq[0,0]=word2idx_targets['<sos>']
  
  eos=word2idx_targets['<eos>']
  output_sentence=[]
  
  for i in range(max_len_target):
    output,h,c=decoder_model.predict(
      [target_seq] + states
)
    
    idx=np.argmax(output[0,0,:])
    if idx==eos:
      break
      
    word=''
    if idx>0:
      word=idx2word_targets[idx]
      output_sentence.append(word)
    target_seq[0,0]=idx
    states=[h,c]
    
    
  return ' '.join(output_sentence)  
      
      
      
      
    
    
  
  
  

In [33]:
while True:
  i=np.random.choice(len(input_texts))
  input_seq=encoder_inputs[i:i+1]
  translation=decode_sequence(input_seq)
  print('Input:', input_texts[i])
  print('Translation:', translation)
  
  ans = input("Continue? [Y/n]")
  if ans and ans.lower().startswith('n'):
     break
  

Input: They know Tom.
Translation: conocen a tom.
Continue? [Y/n]y
Input: Nobody laughed.
Translation: nadie se rió.
Continue? [Y/n]n
